In [37]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import cv2
import numpy as np
from tensorflow.keras.metrics import MeanAbsoluteError

emotion_model_path = "C:\\emotion_detection_model_100epochs.h5"
gender_model_path = "C:\\gender_model_50epochs.h5"
age_model_path = "C:\\age_detection_model_30epochs.h5"
race_model_path =  "C:\\race_detection_model_10epoch.h5"

# Load the models
emotion_model = load_model(emotion_model_path)
gender_model = load_model(gender_model_path)
age_model = load_model(age_model_path, custom_objects={'mse': MeanAbsoluteError()})
race_model = load_model(race_model_path)  # Load the race model

face_classifier = cv2.CascadeClassifier("C:\\haarcascade_frontalface_default.xml")

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
gender_labels = ['Male', 'Female']
race_labels = ['White', 'Black', 'Asian', 'Indian', 'Others']

# Read the image from file
#image_path = "C:\\asian4.jpg"
image_path = "C:\\blackwoman.jpg"

frame = cv2.imread(image_path)

# Convert image to grayscale for face detection
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    
for (x, y, w, h) in faces:
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

    # Prepare image for emotion prediction
    roi = roi_gray.astype('float') / 255.0  # Scale
    roi = img_to_array(roi)
    roi = np.expand_dims(roi, axis=0)  # Expand dims to get it ready for prediction (1, 48, 48, 1)

    preds = emotion_model.predict(roi)[0]  # Yields one hot encoded result for 7 classes
    print(preds)
    label = class_labels[preds.argmax()]  # Find the label
    print(label)
    label_position = (x, y)
    cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Gender prediction
    roi_color = frame[y:y+h, x:x+w]
    roi_color = cv2.resize(roi_color, (200, 200), interpolation=cv2.INTER_AREA)
    gender_predict = gender_model.predict(np.array(roi_color).reshape(-1, 200, 200, 3))
    gender_predict = (gender_predict >= 0.5).astype(int)[:, 0]
    gender_label = gender_labels[gender_predict[0]]
    gender_label_position = (x, y + h + 50)  # 50 pixels below to move the label outside the face
    cv2.putText(frame, gender_label, gender_label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Age prediction
    roi_age = cv2.cvtColor(roi_color, cv2.COLOR_BGR2GRAY)
    roi_age = cv2.resize(roi_age, (128, 128), interpolation=cv2.INTER_AREA)
    roi_age = roi_age.astype('float') / 255.0
    roi_age = img_to_array(roi_age)
    roi_age = np.expand_dims(roi_age, axis=0)
    age_predict = age_model.predict(roi_age)
    age = round(age_predict[0, 0])
    age_label_position = (x + h, y + h)
    cv2.putText(frame, "Age=" + str(age), age_label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
     # Race prediction
    roi_race = cv2.resize(roi_color, (198, 198), interpolation=cv2.INTER_AREA)  # Resize to 198x198
    roi_race = roi_race.astype('float') / 255.0
    roi_race = img_to_array(roi_race)
    roi_race = np.expand_dims(roi_race, axis=0)  # Ensure shape (1, 198, 198, 3)
    race_pred = race_model.predict(roi_race)
    race_label = race_labels[race_pred.argmax()]
    race_label_position = (x, y + h + 150)
    cv2.putText(frame, "Race: " + race_label, race_label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


# Display the image with predictions
cv2.imshow('Emotion, Gender, and Age Detector', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
[0.07675062 0.00117675 0.10941379 0.04500742 0.59303623 0.15763889
 0.01697626]
Neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
